# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/weather_cities.csv")
weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Albany,95,US,1599100533,91,42.60,-73.97,73.99,3.74
1,Kruisfontein,0,ZA,1599100616,38,-34.00,24.73,60.01,1.01
2,Gerāsh,0,IR,1599100706,43,27.67,54.14,73.40,2.42
3,Torbay,90,CA,1599100523,71,47.67,-52.73,55.00,12.75
4,Thinadhoo,100,MV,1599100706,75,0.53,72.93,82.24,13.38


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 70 and 80
narrowed_cities = weather_data.loc[(weather_data["Max Temp"] > 70) & (weather_data["Max Temp"] < 80) & (weather_data["Cloudiness"] == 0), :]
narrowed_cities = narrowed_cities.dropna(how='any')
narrowed_cities.reset_index(inplace=True)
narrowed_cities

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,2,Gerāsh,0,IR,1599100706,43,27.67,54.14,73.40,2.42
1,36,Arraial do Cabo,0,BR,1599100283,89,-22.97,-42.03,70.50,15.64
2,91,Goderich,0,CA,1599100203,81,43.75,-81.72,73.00,2.86
3,103,Beyneu,0,KZ,1599100437,27,45.32,55.20,77.41,7.09
4,172,São João da Barra,0,BR,1599100656,87,-21.64,-41.05,71.01,8.72
5,177,Kiryat Gat,0,IL,1599100279,76,31.61,34.76,78.01,2.91
6,193,Guerrero Negro,0,MX,1599100070,73,27.98,-114.06,73.89,15.50
7,259,Mīrābād,0,AF,1599100744,18,30.44,61.84,77.59,22.21
8,262,São Félix do Xingu,0,BR,1599100744,44,-6.64,-51.99,73.63,1.34
9,322,Jalu,0,LY,1599100615,23,29.03,21.55,79.77,7.94


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_data = narrowed_cities.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_data["Hotel Name"] = ""

# Display the result
hotel_data

,City,Country,Lat,Lng,Hotel Name
0,Gerāsh,IR,27.67,54.14,
1,Arraial do Cabo,BR,-22.97,-42.03,
2,Goderich,CA,43.75,-81.72,
3,Beyneu,KZ,45.32,55.20,
4,São João da Barra,BR,-21.64,-41.05,
5,Kiryat Gat,IL,31.61,34.76,
6,Guerrero Negro,MX,27.98,-114.06,
7,Mīrābād,AF,30.44,61.84,
8,São Félix do Xingu,BR,-6.64,-51.99,
9,Jalu,LY,29.03,21.55,


In [7]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_data.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_data.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Print end of search once searching is completed
print("-------End of Search-------")




Retrieving Results for Index 0: Gerāsh.
Closest hotel in Gerāsh is Chelpasin.
------------
Retrieving Results for Index 1: Arraial do Cabo.
Closest hotel in Arraial do Cabo is Orlanova Hotel.
------------
Retrieving Results for Index 2: Goderich.
Closest hotel in Goderich is Samuels Hotel.
------------
Retrieving Results for Index 3: Beyneu.
Closest hotel in Beyneu is HOTEL "AKNUR".
------------
Retrieving Results for Index 4: São João da Barra.
Closest hotel in São João da Barra is Pousada Kactus.
------------
Retrieving Results for Index 5: Kiryat Gat.
Closest hotel in Kiryat Gat is Desert Gat.
------------
Retrieving Results for Index 6: Guerrero Negro.
Closest hotel in Guerrero Negro is Hotel Los Caracoles.
------------
Retrieving Results for Index 7: Mīrābād.
Missing field/result... skipping.
------------
Retrieving Results for Index 8: São Félix do Xingu.
Closest hotel in São Félix do Xingu is Apart Hotel Rio Xingu.
------------
Retrieving Results for Index 9: Jalu.
Closest hotel

In [8]:
hotel_data

,City,Country,Lat,Lng,Hotel Name
0,Gerāsh,IR,27.67,54.14,Chelpasin
1,Arraial do Cabo,BR,-22.97,-42.03,Orlanova Hotel
2,Goderich,CA,43.75,-81.72,Samuels Hotel
3,Beyneu,KZ,45.32,55.20,"HOTEL ""AKNUR"""
4,São João da Barra,BR,-21.64,-41.05,Pousada Kactus
5,Kiryat Gat,IL,31.61,34.76,Desert Gat
6,Guerrero Negro,MX,27.98,-114.06,Hotel Los Caracoles
7,Mīrābād,AF,30.44,61.84,
8,São Félix do Xingu,BR,-6.64,-51.99,Apart Hotel Rio Xingu
9,Jalu,LY,29.03,21.55,Jalu Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_data.iterrows()]
locations = hotel_data[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))